Functions available from **DataLakeHelperFunctionsNotebook**

|Function|Purpose|
|--|---|
|mount_lake_container|Takes a container name and mounts it to Databricks. Prints out the name of the mount point. |

In [0]:
def mount_lake_container(pAdlsContainerName):
  
    """
    mount_lake_container: Takes a container name and mounts it to Databricks. Prints out the name of the mount point. 
    Key Vault SecretScopeName = "DianGRAndDKeyVault"
    Key Vault Secret with Data Lake Name: dianrandddatalake-storageaccountname
    Key Vault Secret with ClientID = "RandD-ServicePrinciple-ApplicationID"
    Key Vault Secret with ClientSecret = "RandD-ServicePrinciple-Password"
    Key Vault Secret with TenantID = "RandD-ServicePrinciple-TenantID"
    """
    # Python code to mount and access Azure Data Lake Storage Gen2 Account from Azure Databricks with Service Principal and OAuth

    # KeyVault Secret Scope Name
    VarSecretScopeName = "DianGRAndDKeyVault" ##this would be a fixed name we would have a standard for. Ideally there is only one secret scope for automated notebooks

    # Define the variables used for creating connection strings - Data Lake Related
    varAdlsAccountName = dbutils.secrets.get(scope=VarSecretScopeName,key="dianrandddatalake-storageaccountname") # e.g. "dianrandddatalake" --the storage account name itself
    varAdlsContainerName = pAdlsContainerName # e.g. rawdata, bronze, silver, gold, platinum etc.
    varMountPoint = "/mnt/datalake_" + varAdlsContainerName #fixed since we already parameterised the container name. Ensures there is a standard in mount point naming

    # Define the variables that have the names of the secrets in key vault that store the sensitive information we need for the conenction via Service Principle Auth
    VarSecretClientID = "RandD-ServicePrinciple-ApplicationID" #Name of the generic key vault secret contianing the Service Principle Name.
    VarSecretClientSecret = "RandD-ServicePrinciple-Password" #Name of the generic key vault secret contianing the Service Principle Password. 
    VarSecretTenantID = "RandD-ServicePrinciple-TenantID" #Name of the generic key vault secret contianing the Tenant ID.

    # Get the actual secrets from key vault for the service principle
    varApplicationId = dbutils.secrets.get(scope=VarSecretScopeName, key=VarSecretClientID) # Application (Client) ID
    varAuthenticationKey = dbutils.secrets.get(scope=VarSecretScopeName, key=VarSecretClientSecret) # Application (Client) Secret Key
    varTenantId = dbutils.secrets.get(scope=VarSecretScopeName, key=VarSecretTenantID) # Directory (Tenant) ID

    # Using the secrets above, generate the URL to the storage account and the authentication endpoint for OAuth
    varEndpoint = "https://login.microsoftonline.com/" + varTenantId + "/oauth2/token" #Fixed URL for the endpoint
    varSource = "abfss://" + varAdlsContainerName + "@" + varAdlsAccountName + ".dfs.core.windows.net/"

    # Connecting using Service Principal secrets and OAuth
    varConfigs = {"fs.azure.account.auth.type": "OAuth", #standard
               "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider", #standard
               "fs.azure.account.oauth2.client.id": varApplicationId,
               "fs.azure.account.oauth2.client.secret": varAuthenticationKey,
               "fs.azure.account.oauth2.client.endpoint": varEndpoint}

    # Mount ADLS Storage to DBFS only if the directory is not already mounted
    # Mount is generated as a list of all mount points available already via dbutils.fs.mounts()
    # Then it checks the list for the new mount point we are trying to generate.
    if not any(mount.mountPoint == varMountPoint for mount in dbutils.fs.mounts()): 
      dbutils.fs.mount(
        source = varSource,
        mount_point = varMountPoint,
        extra_configs = varConfigs)

    # print the mount point used for troubleshooting
    print("Mount Point: " + varMountPoint)